In [178]:
from bs4 import BeautifulSoup as bs
import requests, json
from datetime import datetime

In [179]:

def get_usd_curreny():
    dt = datetime.today()
    today = dt.strftime('%Y%m%d')

    data = {"dataBody":{"ricInptRootInfo":{"serviceType":"GU","serviceCode":"F3730","nextServiceCode":"","pkcs7Data":"","signCode":"","signData":"","useSign":"","useCert":"","permitMultiTransaction":"","keepTransactionSession":"","skipErrorMsg":"","mode":"","language":"ko","exe2e":"","hideProcess":"","clearTarget":"","callBack":"shbObj.fncF3730Callback","exceptionCallback":"","requestMessage":"","responseMessage":"","serviceOption":"","pcLog":"","preInqForMulti":"","makesum":"","removeIndex":"","redirectUrl":"","preInqKey":"","_multi_transfer_":"","_multi_transfer_count_":"","_multi_transfer_amt_":"","userCallback":"","menuCode":"","certtype":"","fromMulti":"","fromMultiIdx":"","isRule":"N","webUri":"/index.jsp","gubun":"","tmpField2":""},"조회구분":"","조회일자":today,"고시회차":1,"조회일자_display":"","startPoint":"","endPoint":""},"dataHeader":{"trxCd":"RSHRC0213A01","language":"ko","subChannel":"49","channelGbn":"D0"}}
    url = 'https://bank.shinhan.com/serviceEndpoint/httpDigital'

    res = requests.post(url, data=json.dumps(data))


    currency_exchanges = res.json()['dataBody']['R_RIBF3730_1']
    return {'code': currency_exchanges[0]['통화CODE'], 'cur': currency_exchanges[0]['전신환매도환율']}
    #전신환매도환율
    #통화CODE
usd_cur = get_usd_curreny()

In [180]:
usd_cur['cur']

1333.7

In [181]:
import math

def round_off(num, pos=2):
    t = str(num).split('.')
    t[1] = t[1][:pos]
    return float(".".join(t))

In [182]:
# usd_cur = {'code': 'USD', 'cur': '1296.40'}

card_fee = 0

def exCalc(v):
    visafee = v * 0.011;
    visafee = float(round_off(visafee));
    t = round(v * float(usd_cur['cur']))
    card_fee = t * 0.0018
    card_fee = round_off(card_fee, 0)
    krw = ((v + visafee) * float(usd_cur['cur'])) + card_fee
    return (v+visafee, int(krw), card_fee)

exCalc(4.45)

(4.49, 5998, 10.0)

In [183]:
def default_value():
    tar = 5999
    for i in range(600, 400, -1):
        usd = i/100
        usdv, krw, card_fee = exCalc(usd)
        if (krw < 6000):
            return (usd, krw, card_fee, usdv)
        

usd, krw, card_fee, usdv = default_value()
usd, krw, card_fee, usdv

(4.45, 5998, 10.0, 4.49)

In [184]:
import urllib.parse

currenys = []
currencys_list = ["AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]
def get_currency_rate():
    currenys.append({'USD': {'rate': usd_cur['cur'], 'currency_rate': '1.0'}})
    dt = datetime.today()
    today = dt.strftime('%m/%d/%Y')
# 
# https://www.visakorea.com/cmsapi/fx/rates?amount=4.55&fee=2&utcConvertedDate=08%2F14%2F2023&exchangedate=08%2F14%2F2023&fromCurr=KRW&toCurr=USD
    for cur in currencys_list:
        # url = 'https://www.visakorea.com/travel-with-visa/exchange-rate-calculator.html?fromCurr=USD&toCurr={}&fee=0&utcConvertedDate={}&exchangedate={}&submitButton=Calculate+Exchange+Rates'.format(cur, today, today)
        url = 'https://www.visakorea.com/cmsapi/fx/rates?amount={}&fee=0&utcConvertedDate={}&exchangedate={}&fromCurr=USD&toCurr={}'.format(usd, today, today, cur)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        res = requests.get(url, headers=headers)
        currency_rate = res.json()['originalValues']['fxRateVisa']

        currenys.append({cur: {'rate': usd_cur['cur'], 'currency_rate': currency_rate if currency_rate != '' else '1.0'}})
    return currenys


In [185]:
currenys = get_currency_rate()

In [186]:
currenys

[{'USD': {'rate': 1333.7, 'currency_rate': '1.0'}},
 {'AED': {'rate': 1333.7, 'currency_rate': '0.2723308824'}},
 {'ARS': {'rate': 1333.7, 'currency_rate': '0.001945215'}},
 {'AUD': {'rate': 1333.7, 'currency_rate': '0.653599346'}},
 {'CAD': {'rate': 1333.7, 'currency_rate': '0.7457114094'}},
 {'CHF': {'rate': 1333.7, 'currency_rate': '1.1450807287'}},
 {'CLP': {'rate': 1333.7, 'currency_rate': '0.0011715089'}},
 {'CZK': {'rate': 1333.7, 'currency_rate': '0.0457246914'}},
 {'CNY': {'rate': 1333.7, 'currency_rate': '0.1383356931'}},
 {'DKK': {'rate': 1333.7, 'currency_rate': '0.1476994313'}},
 {'EUR': {'rate': 1333.7, 'currency_rate': '1.100698899'}},
 {'GBP': {'rate': 1333.7, 'currency_rate': '1.274198726'}},
 {'HKD': {'rate': 1333.7, 'currency_rate': '0.1280047874'}},
 {'HUF': {'rate': 1333.7, 'currency_rate': '0.0028785233'}},
 {'IDR': {'rate': 1333.7, 'currency_rate': '0.0000657333'}},
 {'INR': {'rate': 1333.7, 'currency_rate': '0.0120905706'}},
 {'JPY': {'rate': 1333.7, 'currency_r

In [187]:
def get_info_with_currency_and_value(cur, v):
    idx = currencys_list.index(cur) + 1
    return exCalc(round(v * float(currenys[idx][cur]['currency_rate']), 2))

In [188]:
exCalc(4.45)

(4.49, 5998, 10.0)

In [189]:
a = get_info_with_currency_and_value('ARS', 25)
a

(0.05, 66, 0.0)

In [190]:
def get_all_info_currency_with_value():
    currencys_list = ["USD", "AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]
    comma = [0,2,0,2,2,2,0,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0]
    usd, krw, card_fee, usdv = default_value()

    for idx, cur in enumerate(currencys_list):
        for i in range(10, 20000000, 1):
            t = i/100
            a = t * float(currenys[idx][cur]['currency_rate']) 
            amount = 0
            if (float("{}{}".format(str(usd), 5)) <= a):                
                prev = t - 0.01
                if (comma[idx] == 0):
                    amount = round_off(prev, 0) 
                else:
                    amount = round(prev, 2)
                currenys[idx][cur]['amount'] = amount if cur != 'USD' else usdv
                currenys[idx][cur]['usd'] = usdv
                currenys[idx][cur]['krw'] = krw
                currenys[idx][cur]['card_fees'] = card_fee
                break
    return currenys

In [191]:
a = get_all_info_currency_with_value()
a

[{'USD': {'rate': 1333.7,
   'currency_rate': '1.0',
   'amount': 4.49,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'AED': {'rate': 1333.7,
   'currency_rate': '0.2723308824',
   'amount': 16.35,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'ARS': {'rate': 1333.7,
   'currency_rate': '0.001945215',
   'amount': 2290.0,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'AUD': {'rate': 1333.7,
   'currency_rate': '0.653599346',
   'amount': 6.81,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'CAD': {'rate': 1333.7,
   'currency_rate': '0.7457114094',
   'amount': 5.97,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'CHF': {'rate': 1333.7,
   'currency_rate': '1.1450807287',
   'amount': 3.89,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'CLP': {'rate': 1333.7,
   'currency_rate': '0.0011715089',
   'amount': 3802.0,
   'usd': 4.49,
   'krw': 5998,
   'card_fees': 10.0}},
 {'CZK': {'rate': 1333.7,
   'currency_rate': '